In [154]:
import numpy as np
from math import sin, cos, radians
import random
from PIL import Image

In [469]:
degrees = {'x' : 0, 'y' : 40, 'z' : -90}
a = {k: radians(v) for k,v in degrees.items()}

height = 1000
width = 1000


z_shift = 0.1
extra_z = 0.0001 + z_shift
init_scale = 7000
scale = init_scale * z_shift
shift = np.array([height / 2, width / 2, 300])
model_shift = np.array([250 / init_scale, 0, 0])

white = np.array([-255, -255, -255])
l = np.array([0, 0, 1])

X_rotate = np.array([[1, 0, 0], [0, cos(a['x']), -sin(a['x'])], [0, sin(a['x']), cos(a['x'])]])
Y_rotate = np.array([[cos(a['y']), 0, sin(a['y'])], [0, 1, 0], [-sin(a['y']), 0, cos(a['y'])]])
Z_rotate = np.array([[cos(a['z']), -sin(a['z']), 0], [sin(a['z']), cos(a['z']), 0], [0, 0, 1]])

In [184]:
def process_obj(filename : str):    
    with open(filename) as obj:
        obj.seek(0, 2)
        length = obj.tell()
        obj.seek(0, 0)

        vertices = []
        faces = []
        
        while obj.tell() != length:
            line = obj.readline().split()
            if line[0] == 'v':
                vertices.append([float(line[1]), float(line[2]), float(line[3])])
            elif line[0] == 'f':
                faces.append([int(line[1].split('/')[0]) - 1, int(line[2].split('/')[0]) - 1, int(line[3].split('/')[0]) - 1])
        
        return np.array(vertices), faces

In [185]:
def barycentric_coordinates(x, y, x0, y0, x1, y1, x2, y2):
    res = []
    res.append(((x - x2) * (y1 - y2) - (x1 - x2) * (y - y2))/((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2)))
    res.append(((x0 - x2) * (y - y2) - (x - x2) * (y0 - y2))/((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2)))
    res.append(1.0 - res[0] - res[1])

    return res

In [186]:
def draw_triangle(image, p0, p1, p2, w, h, color, z_buffer):
    x0 = p0[0]
    y0 = p0[1]
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    
    xmin = int(min(x0, x1, x2))
    xmax = int(max(x0, x1, x2))
    ymin = int(min(y0, y1, y2))
    ymax = int(max(y0, y1, y2))

    xmin = 0 if xmin < 0 else xmin
    ymin = 0 if ymin < 0 else ymin
    xmax = xmax if xmax < w - 1 else w - 1
    ymax = ymax if ymax < h - 1 else h - 1
    
    for i in range(xmin, xmax + 1):
        for j in range(ymin, ymax + 1):
            coords = barycentric_coordinates(i, j, x0, y0, x1, y1, x2, y2)
            z_coord = coords[0] * p0[2] + coords[1] * p1[2] + coords[2] * p2[2]
            if (coords is not None) and coords[0]>=0 and coords[1]>=0 and coords[2]>=0 and z_coord < z_buffer[i, j]:
                z_buffer[i, j] = z_coord
                image[i, j] = color
            

In [187]:
def get_random_color():
    
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return (r, g, b)

In [188]:
def find_norm(p0, p1, p2):
    norm = []
    vx1 = p0[0] - p1[0]
    vy1 = p0[1] - p1[1]
    vz1 = p0[2] - p1[2]
    vx2 = p1[0] - p2[0]
    vy2 = p1[1] - p2[1]
    vz2 = p1[2] - p2[2]

    x = vy1 * vz2 - vz1 * vy2
    y = vz1 * vx2 - vx1 * vz2
    z = vx1 * vy2 - vy1 * vx2

    norm.append(x)
    norm.append(y)
    norm.append(z)

    return norm

In [474]:
def shift_scale(vertices):
    return vertices * init_scale + shift

In [477]:
filename = 'model_1.obj'
vertices, faces = process_obj(filename)
R = Y_rotate @ X_rotate @ Z_rotate
vertices = vertices @ R

vertices += model_shift

min_z = vertices[:, 2].min()

vertices[:, 0] = scale * vertices[:, 0] / (vertices[:, 2] + abs(min_z) + extra_z) + 500
vertices[:, 1] = scale * vertices[:, 1] / (vertices[:, 2] + abs(min_z) + extra_z) + 500
vertices[:, 2] *= 3000

#vertices = shift_scale(vertices)

matrix = np.full(shape=(height, width, 3), fill_value=[0, 0, 0], dtype = np.uint8)
z_buffer = np.full(shape=(height, width), fill_value=np.inf)

print(vertices)

[[420.57362403 299.02583847 -82.18145832]
 [424.42866445 304.5803389  -83.1656648 ]
 [425.1018327  303.32722308 -80.57312421]
 ...
 [665.21197369 346.03371309 -34.57780075]
 [662.01171742 343.54980631 -33.42736997]
 [658.08648631 338.6452685  -25.54044747]]


In [478]:
for triangle in faces:
    norm = find_norm(vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]])
    angle = np.dot(norm, l)/np.linalg.norm(norm)
    if (angle < 0):
        draw_triangle(matrix, vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]], height, width, white * angle, z_buffer)
        
Image.fromarray(matrix, 'RGB').save("draw_rabbit.jpg")